In [6]:
"""
Error Code:
    0: Error opening file
"""

import pandas as pd
import numpy as np

import re
import zipfile

from os import listdir
from os.path import isfile, join

import sqlite3

# Clarify the mission N number for this .ipynb
N_number = 1

# Define a display function for visualization   

In [7]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def display(items):
    print(namestr(items, globals()))
    for item in items:
        print("    " + item)

# Test display
# test_dict = {"A": [1, 2, 3], "B": [4, 5, 6]}
# display(test_dict)

# Define paths and related files (addresses)

In [8]:
mypath = "./"
zip_file_name = "receipt-id-773931-part-001"
zip_file = zipfile.ZipFile(mypath + zip_file_name + ".zip")
file_name_list = zip_file.namelist()
# Should return 4 folders if testing on ASG Zip
print("{0} files detected under current directory. ".format(len(file_name_list)))

53528 files detected under current directory. 


# Filter files by their names

In [17]:
def filter_by_filename(files_list):
    """Function filtering files by filenames.
    Only accepts files starts with "journal-article"
    """
    filtered_list = []
    mask_match = "ngram" + str(N_number) + "/journal-article"
    for filename in files_list:
        # Check if the filename starts with "journal-article"
        assert isinstance(filename, str)
        # Check the first 20 characters of the file name
        if filename.startswith(mask_match, 0, len(mask_match)):
            filtered_list.append(filename)
    return filtered_list

# Run "filter_by_filename" for current directory
filtered_list = filter_by_filename(file_name_list)
print("{0} files remained after filename filtering. ".format(len(filtered_list)))

13382 files remained after filename filtering. 


# Define methods filtering ngram characters

In [18]:
# Functions checking word attributes (single-letter, starts/ends with numebr)
def is_single_letter(word):
    assert isinstance(word, str)
    return len(word) <= 1

def starts_with_number(word):
    assert isinstance(word, str)
    try:
        return word[0].isdigit()
    except:
        return False

def ends_with_number(word):
    assert isinstance(word, str)
    try:
        return word[len(word) - 1].isdigit()
    except:
        return False

# Summary of check functions
check_funcs = [
    is_single_letter, 
    starts_with_number, 
    ends_with_number,
]

# Define method parse ISSN from filename

In [19]:
""" Define function getting article ID from a filename by parsing pattern
Pattern: journal-article-10.2307_977118-ngram1.txt
"""
def parse_id(filename):
    id_number_lst = re.findall("_(.+)-ngram1.txt", filename)
    if len(id_number_lst) == 1:
        return id_number_lst[0]
    print("Parse_ID Error: Filename does not match pattern. ")
    return None

# Test Parsing
# print(filtered_list[0])
# print(parse_id(filtered_list[0]))

# Iterate through the files and clean data (Create DataFrame)

In [23]:
# Run filtering for each file
progress = list(range(0, len(filtered_list), len(filtered_list) // 99))

"""
Data
"""
data = []
for file_num in range(0, len(filtered_list)):
    
    file = filtered_list[file_num]
    # Open file
    try:
        file_open = zip_file.open(file, mode="r")
    except IOError:
        print("Error opening file {0}".format(file))
        exit(0)
    
    # Read by line
    for line in file_open:
        # Line decode using UTF-8
        line = line.decode("utf-8")
        assert isinstance(line, str)
        pair = line.strip().split()
    
        # Separate word/freq
        word, freq = pair
        assert isinstance(word, str)
        
        # Filter by word's attribute
        check_results = [check_func(word) for check_func in check_funcs]
        if any(check_results):
            continue
            
        # Update data for DataFrame
        data.append([word, freq])
    
    # Close reading file
    file_open.close()
    
    # Track Progress
    if len(progress) > 0 and file_num >= progress[0]:
        print("Progress: {}%".format(100 - len(progress)))
        progress.pop(0)
    
"""
Columns
"""
columns = ["word", "freq"]

"""
Index
"""
index = list(range(len(data)))

"""
DataFrame
"""
dataframe = pd.DataFrame(data, columns=columns, index=index)

    
# Print Sample Output
print(dataframe.head()) 

Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progress: 42%
Progress: 43%
Progress: 44%
Progress: 45%
Progress: 46%
Progress: 47%
Progress: 48%
Progress: 49%
Progress: 50%
Progress: 51%
Progress: 52%
Progress: 53%
Progress: 54%
Progress: 55%
Progress: 56%
Progress: 57%
Progress: 58%
Progress: 59%
Progress: 60%
Progress: 61%
Progress: 62%
Progress: 63%
Progress: 64%
Progress: 65%
Progress: 66%
Progress: 67%
Progress: 68%
Progress: 69%
Progress: 70%
Progress: 71%
Pr

# Store Data into SQLite Database

In [25]:
"""
Store data from DataFrame into SQLite
ngram | issn | words | freq |
"""

# Define path towards the database
db_name = "ngram.db"
db_path = join(mypath, db_name)

# Connect to the database
database = sqlite3.connect(db_path)

# Define a cursor to execute commands towards the database
cursor = database.cursor()

# Create table "dataset
cur.execute("CREATE TABLE dataset (n_number, issn, words, freq);")

# This is the qmark style:
cur.execute("INSERT INTO dataset VALUES (?, ?)", (who, age))


          word freq
0  information  131
1     planning  100
2          can   48
3      systems   43
4      complex   41
